In [90]:
import requests
import json
import urllib.parse
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import re
import pandas as pd

In [38]:
def getSoup(link):
    sleep(0.1) #to be kind to the website
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml')

soup = getSoup("https://www.skupina.coop/cooperative/list/")

In [5]:
soup

<!DOCTYPE html>
<html lang="cs">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Adresář členských družstev skupiny COOP | COOP</title>
<meta content="Svaz českých a moravských spotřebních družstev - Skupina COOP. Největší síť maloobchodních prodejen v České republice." name="description"/>
<meta content="COOP,družstva,jednota,TUTY,TIP,TERNO,DISKONT,Stavebniny,Střední školy,maloobchod,potraviny,akce,Europcar,COOP Mobil" name="keywords"/>
<meta content="CloudSailor s.r.o." name="author"/>
<meta content="CloudSailor s.r.o." name="designer"/>
<meta content="index, follow" name="robots"/>
<link href="/temp/cssloader-8859b2325e8a.css?1570101575" rel="stylesheet" type="text/css"/>
<script src="/temp/jsloader-106a2452f059.js?1463559544" type="text/javascript"></script>
<script src="/skins/coop/js/jquery.fancybox-1.3.4.pack.js"></script>
<link href="//cdnjs.cloudflare.com/ajax/lib

In [6]:
names = soup.findAll('h2', {'class':'cooperative'})
names

[<h2 class="cooperative"><a href="/jednota-benesov-druzstvo">JEDNOTA Benešov, družstvo</a></h2>,
 <h2 class="cooperative"><a href="/coop-horovice-druzstvo">COOP Hořovice, družstvo</a></h2>,
 <h2 class="cooperative"><a href="/druzstvo-jednota-kladno">Družstvo JEDNOTA KLADNO</a></h2>,
 <h2 class="cooperative"><a href="/coop-mnichovo-hradiste-druzstvo">COOP Mnichovo Hradiště, družstvo</a></h2>,
 <h2 class="cooperative"><a href="/jednota-spotrebni-druzstvo-v-nymburce">JEDNOTA, spotřební družstvo v Nymburce</a></h2>,
 <h2 class="cooperative"><a href="/coop-praha-zapad">COOP  Praha-západ, družstvo</a></h2>,
 <h2 class="cooperative"><a href="/coop-pribram-druzstvo">Coop Příbram, družstvo</a></h2>,
 <h2 class="cooperative"><a href="/jednota-spotrebni-druzstvo-v-rakovniku">JEDNOTA, spotřební družstvo v Rakovníku</a></h2>,
 <h2 class="cooperative"><a href="/coop-druzstvo-plasy">COOP družstvo Plasy</a></h2>,
 <h2 class="cooperative"><a href="/zapadoceske-konzumni-druzstvo-plzen">Západočeské konzu

In [48]:
def getAllLinks(link):
    soup = getSoup(link)
    h2s = soup.findAll('h2', {'class':'cooperative'})
    return ['https://www.skupina.coop' + h2.find('a')['href'] for h2 in h2s]

links = getAllLinks('https://www.skupina.coop/cooperative/list/')
print(links)

['https://www.skupina.coop/jednota-benesov-druzstvo', 'https://www.skupina.coop/coop-horovice-druzstvo', 'https://www.skupina.coop/druzstvo-jednota-kladno', 'https://www.skupina.coop/coop-mnichovo-hradiste-druzstvo', 'https://www.skupina.coop/jednota-spotrebni-druzstvo-v-nymburce', 'https://www.skupina.coop/coop-praha-zapad', 'https://www.skupina.coop/coop-pribram-druzstvo', 'https://www.skupina.coop/jednota-spotrebni-druzstvo-v-rakovniku', 'https://www.skupina.coop/coop-druzstvo-plasy', 'https://www.skupina.coop/zapadoceske-konzumni-druzstvo-plzen', 'https://www.skupina.coop/jednota-konzumni-druzstvo-rokycany', 'https://www.skupina.coop/zapadoceske-konzumni-druzstvo-susice', 'https://www.skupina.coop/jednota-spotrebni-druzstvo-v-tachove', 'https://www.skupina.coop/coop-jednota-touzim-spotrebni-druzstvo', 'https://www.skupina.coop/spotrebni-druzstvo-jednota-kadan', 'https://www.skupina.coop/druzstvo-jednota-most', 'https://www.skupina.coop/jednota-spotrebni-druzstvo-v-podboranech', 'ht

In [55]:
def getAllSublinks(link):
    list = []
    links = getAllLinks(link)
    for link in links:
        sleep(0.1)
        soup = getSoup(link)
        aas = soup.findAll('a', text = re.compile("detail"))      
        list.extend(['https://www.skupina.coop' + a['href'] for a in aas])
    return list

sublinks = getAllSublinks('https://www.skupina.coop/cooperative/list/')
len(sublinks)

2502

In [7]:
def parse_name(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    return "COOP " + soup.find('h1').find("span").text

parse_name(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))

'COOP Vlašimská 7, Louňovice pod Blaníkem'

In [19]:
def parse_address(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    address = soup.find('h2', text = re.compile("ADRESA")).next_sibling.next_sibling.get_text(separator = ", ").replace("\n","").replace("\t","")
    return address[:(len(address)-2)]
    
parse_address(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))

'Vlašimská 7, 257 06 Louňovice pod Blaníkem, Benešov'

In [21]:
def parse_druzstvo(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    return soup.find('h2', text = re.compile("Družstvo")).next_sibling.next_sibling.text.strip()
    
parse_druzstvo(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))

'JEDNOTA Benešov, družstvo'

In [23]:
def parse_druzstvo_link(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    return 'https://www.skupina.coop' + soup.find('h2', text = re.compile("Družstvo")).next_sibling.next_sibling.find("a")["href"]
    
parse_druzstvo_link(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))

'https://www.skupina.coop/jednota-benesov-druzstvo'

In [58]:
def parse_phone(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    try:
        return soup.find('div',{'class':'kontakty'}).find('li',{'class':'telefon'}).text.strip()
    except:
        return "-"

parse_phone(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))

'317 852 650'

In [30]:
def parse_link(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    try:
        return soup.find('div',{'class':'kontakty'}).find('li',{'class':'link'}).text.strip()
    except:
        return "-"


#parse_link(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))
parse_link(getSoup("https://www.skupina.coop/coop-sobechleby-129-sobechleby-617"))

'www.jednotaostroh.cz'

In [59]:
def parse_mail(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''
    try:
        return soup.find('div',{'class':'kontakty'}).find('li',{'class':'mail'}).text.strip()
    except:
        return "-"

parse_mail(getSoup(parse_druzstvo_link(getSoup("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254"))))

'-'

In [109]:
def parse_coop_shops(link,pause=.5):
    '''
    From URL of given news story generate pd.Series object with `title`, `date` and `content`.

    Use functions `parse_title`, `parse_date` and `parse_news_content` to get individual attributes.

    Please, keep the sleep() to prevent overflow of IES website.
    '''

    sleep(pause)

    soup = getSoup(link)
    soup_dr = getSoup(parse_druzstvo_link(getSoup(link)))
    gps = get_GPS(soup)

    return pd.Series({
        'name':parse_name(soup),
        'address':parse_address(soup),
        'phone':parse_phone(soup),
        'mail':parse_mail(soup),
        'webpage':parse_link(soup),
        'druzstvo':parse_druzstvo(soup),
       # 'address_dr':parse_address(soup_dr),
        'phone_dr':parse_phone(soup_dr),
        'mail_dr':parse_mail(soup_dr),
        'webpage_dr':parse_link(soup_dr),
        'lat':gps[0],
        'lng':gps[1]
    })

parse_coop_shops("https://www.skupina.coop/coop-vlasimska-7-lounovice-pod-blanikem-254")

name                   COOP Vlašimská 7, Louňovice pod Blaníkem
address       Vlašimská 7, 257 06 Louňovice pod Blaníkem, Be...
phone                                               317 852 650
mail                                                          -
webpage                                                       -
druzstvo                              JEDNOTA Benešov, družstvo
phone_dr                                            317 719 111
mail_dr                                                       -
webpage_dr                                                    -
lat                                                     49.6385
lng                                                     14.8475
dtype: object

In [110]:
def get_all_shops(link):
    '''
    wraping fuctions that accepts a `link` pointing towards hub website with links to parse and returing a dataframe containing all the links
    '''
    links = getAllSublinks(link)[:2]

    return pd.DataFrame([parse_coop_shops(link) for link in links])

shops = get_all_shops('https://www.skupina.coop/cooperative/list/')
print(shops)

                                       name  \
0  COOP Vlašimská 7, Louňovice pod Blaníkem   
1                   COOP Husova 471, Votice   

                                             address        phone mail  \
0  Vlašimská 7, 257 06 Louňovice pod Blaníkem, Be...  317 852 650    -   
1                 Husova 471, 259 01 Votice, Benešov  317 812 530    -   

  webpage                   druzstvo     phone_dr mail_dr webpage_dr  \
0       -  JEDNOTA Benešov, družstvo  317 719 111       -          -   
1       -  JEDNOTA Benešov, družstvo  317 719 111       -          -   

         lat        lng  
0  49.638473  14.847524  
1  49.638246  14.636691  


In [106]:
def get_GPS(soup):
    key = "XU4Bdq1mMc6cN6d6RAGSJpVA0GdKGVIHkx"
    adr = urllib.parse.quote_plus(parse_address(soup))
    r = requests.get("https://www.mapquestapi.com/geocoding/v1/address?key={}&inFormat=kvp&outFormat=json&location={}&thumbMaps=false".format(key,adr))
    d = r.json()
    return [d["results"][0]["locations"][0]["latLng"]["lat"],d["results"][0]["locations"][0]["latLng"]["lng"]]

14.847524